# Hybrid
Now SLIMNet and RP3 are my best models, let's try to combine them since they have the same characteristics for model merging.

In [1]:
import os

os.chdir('../')

## Some analysis

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


# Building Hybrid

In [8]:
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender_slimNet = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)
recommender_slimNet.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)

recommender_rp3 = RP3betaRecommender(URM_train_validation)
recommender_rp3.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)

Progress: 100%|██████████| 38121/38121 [08:56<00:00, 71.03items/s]


RP3betaRecommender: Similarity column 38121 (100.0%), 3846.02 column/sec. Elapsed time 9.91 sec


In [9]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function(trial):

    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


    recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

    new_similarity = recommender_slimNet.W_sparse * alpha + recommender_rp3.W_sparse * (1-alpha)

    recommender.fit(new_similarity)

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [10]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_RP3_MM.csv", index = False)

In [11]:
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_ep3", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-24 17:48:49,644] A new study created in memory with name: hybrid_slim_ep3
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3036968644.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.36 sec. Users per second: 2317


[I 2024-12-24 17:49:05,061] Trial 0 finished with value: 0.05703360223145108 and parameters: {'alpha': 0.17934580697435976}. Best is trial 0 with value: 0.05703360223145108.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3036968644.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.30 sec. Users per second: 2326


[I 2024-12-24 17:49:20,407] Trial 1 finished with value: 0.060279897078462354 and parameters: {'alpha': 0.9369286807355108}. Best is trial 1 with value: 0.060279897078462354.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3036968644.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


...

[I 2024-12-24 18:13:40,639] Trial 97 finished with value: 0.060756183430433826 and parameters: {'alpha': 0.7216088095387635}. Best is trial 90 with value: 0.0607693919914721.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3036968644.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.24 sec. Users per second: 2335


[I 2024-12-24 18:13:55,923] Trial 98 finished with value: 0.060719954648524435 and parameters: {'alpha': 0.7581978755902727}. Best is trial 90 with value: 0.0607693919914721.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3036968644.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35595 (100.0%) in 15.21 sec. Users per second: 2341


[I 2024-12-24 18:14:11,166] Trial 99 finished with value: 0.060748196754937735 and parameters: {'alpha': 0.682194048322318}. Best is trial 90 with value: 0.0607693919914721.


In [12]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.0607693919914721, 'alpha': 0.7161276393044346}


Let's compare it to our best model

In [13]:
alpha = 1

recommender = ItemKNNCustomSimilarityRecommender(URM_train_validation)

new_similarity = recommender_slimNet.W_sparse * alpha + recommender_rp3.W_sparse * (1-alpha)

recommender.fit(new_similarity)

result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

EvaluatorHoldout: Processed 35595 (100.0%) in 15.37 sec. Users per second: 2316
0.06001860882012401


So we had a 0.01 improvement

# Cross Validation

In [14]:
k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]
assert len(seeds) == k_fold

In [15]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()
urm_trains = []
urm_validations = []
urm_test = []
urm_train_validations = []
# Defining evaluators
evaluator_validation = []
evaluator_test = []

for i in range(k_fold):
    # Splitting data into 64% train, 16% validation, 20% test
    URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=seeds[i])
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8, seed=seeds[i])
    
    urm_trains.append(URM_train)
    urm_validations.append(URM_validation)
    urm_test.append(URM_test)
    urm_train_validations.append(URM_train_validation)
    
    evaluator_validation.append(EvaluatorHoldout(URM_validation, cutoff_list=[10]))
    evaluator_test.append(EvaluatorHoldout(URM_test, cutoff_list=[10]))

EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 447 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 155 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 441 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 467 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 169 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 413 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 154 ( 0.4%) Users that have less than 

In [ ]:
slim_recommenders = []
rp3_recommenders = []

for i in range(k_fold):
    recommender_slimNet = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validations[i])
    recommender_slimNet.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)
    slim_recommenders.append(recommender_slimNet)
    
    recommender_rp3 = RP3betaRecommender(urm_train_validations[i])
    recommender_rp3.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)
    rp3_recommenders.append(recommender_rp3)

## Now again optuna

In [26]:
def objective_function(trial):
    alpha = trial.suggest_uniform('alpha', 0.0, 1.0)

    MAP = 0
    for i in range(k_fold):
        recommender = ItemKNNCustomSimilarityRecommender(urm_train_validations[i])

        new_similarity = slim_recommenders[i].W_sparse * alpha + rp3_recommenders[i].W_sparse * (1-alpha)

        recommender.fit(new_similarity)

        result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

        MAP += result_dict.loc[10]["MAP"]
    
    MAP /= k_fold
    
    return MAP

In [27]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/Hybrid/hybrid_slim_RP3_MM_CROSSVALIDATIOn.csv", index = False)

In [28]:
import optuna

optuna_study = optuna.create_study(study_name="hybrid_slim_rp3_CROSSVAL", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-24 21:04:07,733] A new study created in memory with name: hybrid_slim_rp3_CROSSVAL
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3888098177.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35569 (100.0%) in 15.39 sec. Users per second: 2311
EvaluatorHoldout: Processed 35581 (100.0%) in 15.43 sec. Users per second: 2305
EvaluatorHoldout: Processed 35569 (100.0%) in 15.42 sec. Users per second: 2307
EvaluatorHoldout: Processed 35573 (100.0%) in 15.32 sec. Users per second: 2323
EvaluatorHoldout: Processed 35567 (100.0%) in 15.64 sec. Users per second: 2274
EvaluatorHoldout: Processed 35582 (100.0%) in 15.22 sec. Users per second: 2338
EvaluatorHoldout: Processed 35567 (100.0%) in 15.19 sec. Users per second: 2341
EvaluatorHoldout: Processed 35578 (100.0%) in 15.24 sec. Users per second: 2334
EvaluatorHoldout: Processed 35563 (100.0%) in 15.23 sec. Users per second: 2334
EvaluatorHoldout: Processed 35560 (100.0%) in 15.21 sec. Users per second: 2338


[I 2024-12-24 21:06:41,786] Trial 0 finished with value: 0.059304029021660834 and parameters: {'alpha': 0.29126618073263155}. Best is trial 0 with value: 0.059304029021660834.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3888098177.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35569 (100.0%) in 15.21 sec. Users per second: 2339
EvaluatorHoldout: Processed 35581 (100.0%) in 15.17 sec. Users per second: 2345
EvaluatorHoldout: Processed 35569 (100.0%) in 15.18 sec. Users per second: 2343
EvaluatorHoldout: Processed 35573 (100.0%) in 15.19 sec. Users per second: 2342
EvaluatorHoldout: Processed 35567 (100.0%) in 15.22 sec. Users per second: 2337
EvaluatorHoldout: Processed 35582 (100.0%) in 15.25 sec. Users per second: 2332
EvaluatorHoldout: Processed 35567 (100.0%) in 15.14 sec. Users per second: 2349
EvaluatorHoldout: Processed 35578 (100.0%) in 15.24 sec. Users per second: 2335
EvaluatorHoldout: Processed 35563 (100.0%) in 15.17 sec. Users per second: 2344
EvaluatorHoldout: Processed 35560 (100.0%) in 15.19 sec. Users per second: 2341


[I 2024-12-24 21:09:14,101] Trial 1 finished with value: 0.06126806468861749 and parameters: {'alpha': 0.8768724039190811}. Best is trial 1 with value: 0.06126806468861749.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3888098177.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


...

[I 2024-12-25 01:12:14,492] Trial 97 finished with value: 0.061426577616172484 and parameters: {'alpha': 0.8039607247369135}. Best is trial 72 with value: 0.06147879852097141.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3888098177.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35569 (100.0%) in 15.19 sec. Users per second: 2342
EvaluatorHoldout: Processed 35581 (100.0%) in 15.15 sec. Users per second: 2349
EvaluatorHoldout: Processed 35569 (100.0%) in 15.22 sec. Users per second: 2337
EvaluatorHoldout: Processed 35573 (100.0%) in 15.22 sec. Users per second: 2338
EvaluatorHoldout: Processed 35567 (100.0%) in 15.20 sec. Users per second: 2341
EvaluatorHoldout: Processed 35582 (100.0%) in 15.17 sec. Users per second: 2345
EvaluatorHoldout: Processed 35567 (100.0%) in 15.19 sec. Users per second: 2341
EvaluatorHoldout: Processed 35578 (100.0%) in 15.21 sec. Users per second: 2339
EvaluatorHoldout: Processed 35563 (100.0%) in 15.21 sec. Users per second: 2339
EvaluatorHoldout: Processed 35560 (100.0%) in 15.16 sec. Users per second: 2346


[I 2024-12-25 01:14:46,773] Trial 98 finished with value: 0.061463638505055776 and parameters: {'alpha': 0.7651316264947496}. Best is trial 72 with value: 0.06147879852097141.
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_49684/3888098177.py:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 1.0)


EvaluatorHoldout: Processed 35569 (100.0%) in 15.22 sec. Users per second: 2338
EvaluatorHoldout: Processed 35581 (100.0%) in 15.27 sec. Users per second: 2331
EvaluatorHoldout: Processed 35569 (100.0%) in 15.21 sec. Users per second: 2338
EvaluatorHoldout: Processed 35573 (100.0%) in 15.17 sec. Users per second: 2345
EvaluatorHoldout: Processed 35567 (100.0%) in 15.22 sec. Users per second: 2337
EvaluatorHoldout: Processed 35582 (100.0%) in 15.21 sec. Users per second: 2339
EvaluatorHoldout: Processed 35567 (100.0%) in 15.22 sec. Users per second: 2338
EvaluatorHoldout: Processed 35578 (100.0%) in 15.15 sec. Users per second: 2348
EvaluatorHoldout: Processed 35563 (100.0%) in 15.19 sec. Users per second: 2342
EvaluatorHoldout: Processed 35560 (100.0%) in 15.17 sec. Users per second: 2344


[I 2024-12-25 01:17:19,144] Trial 99 finished with value: 0.0614730780702236 and parameters: {'alpha': 0.7358312418781159}. Best is trial 72 with value: 0.06147879852097141.


In [29]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.06147879852097141, 'alpha': 0.7414857378820595}


# Submitting

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [33]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

In [34]:
slim_URM_all = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
slim_URM_all.fit(topK=142, alpha = 9.521340863590419e-05, l1_ratio = 0.4033590645217344)

rp3_URM_all = RP3betaRecommender(URM_all)
rp3_URM_all.fit(topK=14, alpha=0.3683550822991944, beta=0.19877125816137325, implicit=True)

Progress: 100%|██████████| 38121/38121 [11:59<00:00, 52.98items/s] 


RP3betaRecommender: Similarity column 38121 (100.0%), 3454.65 column/sec. Elapsed time 11.03 sec


In [35]:

alpha = 0.7414857378820595
new_similarity = slim_URM_all.W_sparse * alpha + rp3_URM_all.W_sparse * (1-alpha)

final_recommender = ItemKNNCustomSimilarityRecommender(URM_all)
final_recommender.fit(new_similarity)

In [37]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(final_recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/hybrid_slim_rp3_alpha:{alpha}.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


In [38]:
# Just out of curiosity
alpha = 0.7414857378820595
new_similarity = slim_URM_all.W_sparse * alpha + rp3_URM_all.W_sparse * (1-alpha)

final_recommender.fit(new_similarity)

target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(final_recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/hybrid_slim_rp3_Oldalpha:{alpha}.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
